In [ ]:
%matplotlib inline
from __future__ import division
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict

import plotly
import plotly.offline as py
import plotly.graph_objs as go
import plotly.tools as tls
from plotly import __version__
from plotly.offline.offline import _plot_html

import itertools

from IPython.core.display import display, HTML
from os import environ

import warnings
warnings.filterwarnings('ignore')

py.init_notebook_mode() # graphs charts inline
course_id = environ['current_course']

In [ ]:
display(HTML('<title></title><h1><a href="" target=_blank">Home</a> / '+course_id+'</h1>'))

# update courses with graded_problems other than multiple choices
not_mc_list = [

]
# update professional education courses
pe_list = [

]

This is a course-specific dashboard. The graphs show information related to course structure and effectiveness of the course design and structure, based on student success in the course.

#### Table of Contents
[1. Course Structure](#Course-structure-and-activity)

[2. Student Engagement by Content Type](#Video-activity)

[3. Most and Least Successful Problems](#Ten-graded-problems-with-lowest-success-rate)

In [ ]:
def query_moduleActivity(course_id=course_id):
    """
    Give the course_id, query # students with any activity, # students attempted any problem 
    and # students watched any video for each chapter of the course
    """
    # query # students who attempted any problem for each chapter of the course, exclude those with fewer than 20 attempts
    query = """
    Select course_id, sub.index As index, module_id, chapter_name, exact_count_distinct(user_id) As tried_problem
    From
    (SELECT p.course_id As course_id, p.user_id As user_id, c2.index As index, 
    c2.module_id As module_id, c2.name As chapter_name
    FROM [{0}.problem_analysis] p
    Left Join [{0}.course_axis] c1
    on p.problem_url_name = c1.url_name
    Left Join [{0}.course_axis] c2
    On c1.chapter_mid = c2.module_id) sub
    Group By course_id, index, module_id, chapter_name
    Order By index""".format(course_id)
    tried_problem = pd.io.gbq.read_gbq(query, project_id='yourprojectname', verbose=False)
    tried_problem = tried_problem[tried_problem.tried_problem > 20]

    # query # students who watched any video for each chapter of the course, exclude those with fewer than 20 views
    query = """
    Select course_id, index, module_id, chapter_name, exact_count_distinct(username) As watched_video
    From
    (SELECT c1.course_id As course_id, v.username As username, c2.index As index, 
    c2.module_id As module_id, c2.name As chapter_name
    FROM [{0}.video_stats_day] v
    Left Join [{0}.course_axis] c1
    on v.video_id = c1.url_name
    Left Join [{0}.course_axis] c2
    On c1.chapter_mid = c2.module_id) sub
    Group By course_id, index, module_id, chapter_name
    Order By index""".format(course_id)
    try:
        watched_video = pd.io.gbq.read_gbq(query, project_id='yourprojectname', verbose=False)
        watched_video = watched_video[watched_video.watched_video > 20]
    except:
        pass
        

    # query # students with any activity for each chapter of the course, excluding those with fewer than 20 active students
    query = """
    Select sub.course_id As course_id, sub.module_id As module_id, 
    c.name As chapter_name, c.index As index, sub.nactive As nactive
    From [{0}.course_axis] c
    Join 
    (Select course_id As course_id, Regexp_replace(module_id,'i4x://', '') As module_id, 
    exact_count_distinct(student_id) As nactive
    From [{0}.studentmodule]
    Where module_type = 'chapter' 
    Group By course_id, module_id) sub
    On sub.module_id = c.module_id
    Order By index""".format(course_id)
    nactive = pd.io.gbq.read_gbq(query, project_id='yourprojectname', verbose=False)
    nactive = nactive[nactive.nactive > 20]
    
    # merge watched_video, tried_problem, nactive 
    try:
        module_activity = watched_video.merge(tried_problem, how='outer').merge(nactive, how='outer').fillna(0)
    except:
        module_activity = tried_problem.merge(nactive, how='outer').fillna(0)
    return module_activity[module_activity.chapter_name != 0].sort_values('index').set_index('chapter_name')
    
module_activity = query_moduleActivity()
# create a list of course indices to make sure courses are ordered in the visualization
indices = module_activity.index

In [ ]:
def query_cs(course_id = course_id):    
    """
    Given course_id (e.g. 'UBCx__Climate1x__1T2016'), 
    return a list of all the course items (graded_problem, self_test, video, assignment, chapter) 
    from course_axis table ordered by index.
    IMPORTANT: Need to update course_axis in SPD1x first (delete items that belong to SPD2x and SPD3x)
    """
    query = """
    SELECT
    Case 
         When c1.category='problem' And c1.graded='true' Then 'graded_problem'
         When c1.category='problem' And c1.graded!='true' Then 'self_test' 
         Else c1.category
    End As category, c1.index As index, c1.name As name,
    c1.url_name As url_name, c2.name As chapter
    FROM [[{0}.course_axis] c1
    Left Join [{0}.course_axis] c2
    On c1.chapter_mid = c2.module_id
    Where c1.category in ('video', 'problem', 'openassessment', 'chapter')
    Order By c1.index""".format(course_id)

    structure = pd.io.gbq.read_gbq(query, project_id='yourprojectname', verbose=False)
        
    query = """
    Select problem_url_name, exact_count_distinct(item.answer_id) As num
    From [{0}.problem_analysis]
    Group By problem_url_name""".format(course_id)
    nQuestions = pd.io.gbq.read_gbq(query, project_id='yourprojectname', verbose=False)
    structure = structure.merge(nQuestions, left_on='url_name', right_on='problem_url_name', how='left')\
    .drop('problem_url_name', axis=1)
    structure.num = structure.num.fillna(1)
    return structure

course_structure = query_cs()

In [ ]:
# count # of videos, graded_problems, self_test and assignments for each chapter and order the courses
cs_chapter = course_structure.groupby(['chapter', 'category']).num.sum().unstack('category')\
.reindex(indices).dropna(how='all')

cols = []
# some courses do not have all component types
for col in ['video', 'graded_problem', 'self_test', 'openassessment']:
    if col in cs_chapter.columns.values:
        cols.append(col)
cs_chapter = cs_chapter[cols]
cs_chapter['chapter'] = np.nan
cs_chapter.fillna(0, inplace=True)

In [ ]:
def rolling_count(df):
    df['block'] = (df['category'] != df['category'].shift(1)).astype(int).cumsum()
    df['count'] = df.groupby('block').num.cumsum()
    return df
# count # of times an item (graded_problem, self_test, video) appears consecutively
df = course_structure.fillna(method='bfill')
df = df.groupby('chapter').apply(rolling_count)
idx = df.groupby(['chapter', 'block'])['count'].transform(max) == df['count']

### Course structure and activity

#### Course structure

In [ ]:
module_activity = module_activity.reindex(cs_chapter.index)
fig = tls.make_subplots(rows=1, cols=1, print_grid=False, subplot_titles=('', ''))

colors = {'video': 'rgb(106,61,154)', 'graded_problem': 'rgb(31,120,180)', 
          'self_test': 'rgb(51,160,44)', 'openassessment': 'rgb(255,127,0)', 'chapter': 'rgb(0, 0, 0)'}

# traces for module activity
for i in range(0, cs_chapter.shape[1]):
    column_label = cs_chapter.columns[i]
    if cs_chapter.columns[i]=='self_test':
        column_label = 'self test'
    if cs_chapter.columns[i]=='openassessment':
        column_label = 'open assessment'        
    if cs_chapter.columns[i]=='graded_problem':
        column_label = 'graded problem'                
    fig.append_trace(go.Bar(y=cs_chapter.index, x=cs_chapter.iloc[:, i], orientation='h',
                       marker=dict(color=colors[cs_chapter.columns[i]]), name=column_label), 1, 1)

fig['layout']['yaxis1'].update(tickfont=dict(size=8), showgrid=False, autorange='reversed', title="Modules")
fig['layout']['xaxis1'].update(showgrid=False, title='Number of each content type')
fig['layout']['legend'].update(x=1, y=0, traceorder='normal')
fig['layout'].update(height=50+30*len(cs_chapter), width=850, margin=go.Margin(l=185, t=25, b=40), barmode='stack')
print('How many different types of content do I have in my course?' + '\n')
print('This graph shows, for each module (e.g. Final Exam) the number of component types (e.g. video)' + '\n'
      + 'in that module), across all modules in the course.')
config = {"displayModeBar": True, "showLink": False, "displaylogo": False, "modeBarButtonsToRemove": ['sendDataToCloud', 'zoomIn2d', 'zoomOut2d', 'autoScale2d', 'resetScale2d', 'hoverClosestCartesian', 'hoverCompareCartesian']}
plot_htmla, plotdivid, width, height = py.offline._plot_html(fig, config, True, 525, 525, False)
plot_htmla, plotdivid, width, height = py.offline._plot_html(fig, config, True, 525, 525, False)
py.iplot(fig)
display(HTML('<a href="#" style="float: right;">Read more...</a>'))
display(HTML('<a href="#Table-of-Contents" style="float: left;">Back to top</a><a href="" target=_blank style="float: right;">Compare with other courses</a>'))

In [ ]:
def course_item3(course_id=course_id):
    """
    Given course_id, query students' event for video, graded_problem, 
    self_test, openassessment and chapter from the studentmodule table
    => the numbers are slightly different from thosed queried from person_item and video_stats_day
    """
    query = """
    SELECT sub.module_id As module_id, c.index As index, name, 
    c.category As category, nactive
    FROM [yourprojectname:{0}.course_axis] c
    Join 
    (Select Regexp_replace(module_id,'i4x://', '') As module_id, exact_count_distinct(student_id) As nactive
    From [yourprojectname:{0}.studentmodule]
    Where module_type='chapter'
    Group By module_id) sub
    On sub.module_id = c.module_id
    Order By index""".format(course_id)
    df = pd.io.gbq.read_gbq(query, project_id='yourprojectname', verbose=False)
    return df

courseItem3 = course_item3()
# exclude course items accessed by fewer than 20 students 
courseItem3 = courseItem3[courseItem3.nactive > 20].reset_index(drop=True)

def course_item2(course_id=course_id):
    """
    Given course_id, query students' event for video, graded_problem, 
    self_test, openassessment and chapter from the studentmodule table
    => the numbers are slightly different from thosed queried from person_item and video_stats_day
    """
    query = """
    SELECT sub.module_id As module_id, c.index As indexx, name, 
    c.category As category, nactive
    FROM [yourprojectname:{0}.course_axis] c
    Join 
    (Select Regexp_replace(module_id,'i4x://', '') As module_id, exact_count_distinct(student_id) As nactive
    From [yourprojectname:{0}.studentmodule]
    Where module_type='openassessment'
    Group By module_id) sub
    On sub.module_id = c.module_id
    Order By indexx""".format(course_id)
    df = pd.io.gbq.read_gbq(query, project_id='yourprojectname', verbose=False)
    return df

courseItem2 = course_item2()
# exclude course items accessed by fewer than 20 students 
courseItem2 = courseItem2[courseItem2.nactive > 20].reset_index(drop=True)

def course_item(course_id=course_id):
    """
    Given course_id, query students' event for video, graded_problem, 
    self_test, openassessment and chapter from the studentmodule table
    => the numbers are slightly different from thosed queried from person_item and video_stats_day
    """
    query = """
    SELECT sub.module_id As module_id, c.index As index, name, 
    Case 
     When c.category='problem' And c.graded='true' Then 'graded_problem'
     When c.category='problem' And c.graded!='true' Then 'self_test' 
     Else c.category
    End As category, nactive
    FROM [yourprojectname:{0}.course_axis] c
    Join 
    (Select Regexp_replace(module_id,'i4x://', '') As module_id, exact_count_distinct(student_id) As nactive
    From [yourprojectname:{0}.studentmodule]
    Where (module_type='video' And state Not Like '%00:00:00%') Or 
    (module_type='problem' And state Like '%student_answers%') Or module_type In ('openassessment', 'chapter')
    Group By module_id) sub
    On sub.module_id = c.module_id
    Order By index""".format(course_id)
    df = pd.io.gbq.read_gbq(query, project_id='yourprojectname', verbose=False)
    return df

courseItem = course_item()
# exclude course items accessed by fewer than 20 students 
courseItem = courseItem[courseItem.nactive > 20].reset_index(drop=True)

#### Overall learner engagement vs. Course structure

In [ ]:
colors = {'video': 'rgb(106,61,154)', 'graded_problem': 'rgb(31,120,180)', 
          'self_test': 'rgb(51,160,44)', 'openassessment': 'rgb(255,127,0)', 'chapter': 'rgb(0, 0, 0)'}
# make it center in the middle
trace1 = go.Bar(x=courseItem.index+1, y=courseItem.nactive, hoverinfo='text',
               text=['{0}:<br>number of students: {1}'.format(name.encode('utf-8'), value) 
                      for name, value in zip(courseItem.name, courseItem.nactive)],
               marker=dict(color=courseItem.category.apply(lambda x: colors[x]).values))
trace2 = go.Bar(x=courseItem.index+1, y=-courseItem.nactive, hoverinfo='none',
               marker=dict(color=courseItem.category.apply(lambda x: colors[x]).values))

data = [trace1, trace2]
layout = go.Layout(barmode='relative', title='', 
                   xaxis=dict(showticklabels=False, title='Course structure'), 
                   yaxis=dict(showticklabels=False, showgrid=False, title='Number of learners engaging', zeroline=False), 
                   height=350, width=850, margin=go.Margin(t=25, b=25), showlegend=False)
fig = go.Figure(data=data, layout=layout)

print('What is the overall structure of my course? How many learners are engaging with different parts of my course?' + '\n')
print('This "Eiffel Tower" graph shows, for each component type (e.g. video) in each course module (e.g. Part I' + '\n'
      + 'Conclusion), the number of students who interacted with that component. For more information, please see below:')
config = {"displayModeBar": True, "showLink": False, "displaylogo": False, "modeBarButtonsToRemove": ['sendDataToCloud', 'zoomIn2d', 'zoomOut2d', 'autoScale2d', 'resetScale2d', 'hoverClosestCartesian', 'hoverCompareCartesian']}
plot_html, plotdivid, width, height = py.offline._plot_html(fig, config, True, 525, 525, False)
display(HTML(plot_html))
display(HTML('<a href="#" style="float: right;">Read more...</a>'))
display(HTML('<a href="#Table-of-Contents" style="float: left;">Back to top</a><a href="" target=_blank style="float: right;">Compare with other courses</a>'))

In [ ]:
def query_nstudents(cs = course_structure, course_id = course_id):
    """
    Query and calculate number of students viewed the video, attempted the problem
    """
    query = """
    Select vv.mm as mmm, l.video_id As video_id, nstudents, position As length
    From (Select video_id, position, Row_number() Over (Partition By video_id Order By position Desc) As rn
    From [{0}.video_stats_day]) l
    Join 
    (Select video_id, exact_count_distinct(username) As nstudents
    From [{0}.video_stats_day]
    Where position != 0
    Group By video_id) n
    On l.video_id = n.video_id
    
    Join 
    (Select max(position) as mm, video_id from [{0}.video_stats_day]
    group by video_id) vv
    
    On l.video_id = vv.video_id    
    
    Where rn=5""".format(course_id)
    try:
        videos = pd.io.gbq.read_gbq(query, project_id='yourprojectname', verbose=False)
        videos = cs[cs.category=='video'].merge(videos, left_on='url_name', right_on='video_id')
        videos = videos[(videos.length<1800) & (videos.nstudents>10)].reset_index(drop=True)
    except:
        videos = None

    query = """
    SELECT c.chapter_name As chapter, c.section_name As section_name,
    p.item_short_id As name, c.item_id As problem_id,
    count(*) As nstudents, Sum(item_grade) As ncorrect
    FROM [{0}.person_item] p
    Join [{0}.course_item] c
    On p.item_short_id = c.item_short_id
    Group By chapter, section_name, name, c.item_nid, problem_id
    Order By c.item_nid""".format(course_id)

    graded_problems = pd.io.gbq.read_gbq(query, project_id='yourprojectname', verbose=False)
    graded_problems = graded_problems[graded_problems.ncorrect != 0].reset_index(drop=True)
    
    query = """
    SELECT ca.name As name, ca.index As index, ca.url_name As problem_id,
    count(*) As nstudents, Sum(grade) As ncorrect
    FROM [{0}.course_axis] ca
    Join [{0}.problem_analysis] pa
    On pa.problem_url_name = ca.url_name        
    Where ca.graded!='true' AND ca.category='problem'
    Group By name, index, problem_id
    Order By index""".format(course_id)

    self_test = pd.io.gbq.read_gbq(query, project_id='yourprojectname', verbose=False)
    self_test = self_test[self_test.ncorrect != 0].reset_index(drop=True)    
    '''query = """
    SELECT v.video_id As vi, v.length As vl,
    FROM [{0}.video_axis] va""".format(course_id)

    graded_problems = pd.io.gbq.read_gbq(query, project_id='yourprojectname', verbose=False)
    graded_problems = graded_problems[graded_problems.ncorrect != 0].reset_index(drop=True)'''
    
    return videos, graded_problems, self_test

In [ ]:
videos, problems, self_test = query_nstudents(cs = course_structure)

### Video activity

In [ ]:
choices = ['rgba(202,178,214, 0.8)', 'rgba(202,178,214, 0.8)', 'rgba(202,178,214, 0.8)', 'rgba(202,178,214, 0.8)', 
           'rgba(202,178,214, 0.8)', 'rgba(202,178,214, 0.8)', 'rgba(202,178,214, 0.8)', 'rgba(202,178,214, 0.8)', 
           'rgba(202,178,214, 0.8)', 'rgba(202,178,214, 0.8)', 'rgba(202,178,214, 0.8)', 'rgba(202,178,214, 0.8)'
          , 'rgba(202,178,214, 0.8)', 'rgba(202,178,214, 0.8)']
# create a dictionary to map colors to chapters
colors_chapter = dict(zip(cs_chapter.index, choices[:len(cs_chapter.index)]))

choices2 = ['rgba(106,61,154, 0.8)', 'rgba(106,61,154, 0.8)', 'rgba(106,61,154, 0.8)', 'rgba(106,61,154 0.8)', 
           'rgba(106,61,154, 0.8)', 'rgba(106,61,154, 0.8)', 'rgba(106,61,154, 0.8)', 'rgba(106,61,154, 0.8)', 
           'rgba(106,61,154, 0.8)', 'rgba(106,61,154, 0.8)']
# create a dictionary to map colors to chapters, for dataset of full videos lengths
colors_chapter2 = dict(zip(cs_chapter.index, choices2[:len(cs_chapter.index)]))

choices3 = ['rgba(178,223,138, 0.8)', 'rgba(178,223,138, 0.8)', 'rgba(178,223,138, 0.8)', 'rgba(178,223,138, 0.8)', 
           'rgba(178,223,138, 0.8)', 'rgba(178,223,138, 0.8)', 'rgba(178,223,138, 0.8)', 'rgba(178,223,138, 0.8)', 
           'rgba(178,223,138, 0.8)', 'rgba(178,223,138, 0.8)']
# create a dictionary to map colors to chapters
colors_chapter3 = dict(zip(cs_chapter.index, choices3[:len(cs_chapter.index)]))

choices4 = ['rgba(51,160,44, 0.8)', 'rgba(51,160,44, 0.8)', 'rgba(51,160,44, 0.8)', 'rgba(51,160,44, 0.8)', 
           'rgba(51,160,44, 0.8)', 'rgba(51,160,44, 0.8)', 'rgba(51,160,44, 0.8)', 'rgba(51,160,44, 0.8)', 
           'rgba(51,160,44, 0.8)', 'rgba(51,160,44, 0.8)']
# create a dictionary to map colors to chapters, for dataset of full videos lengths
colors_chapter4 = dict(zip(cs_chapter.index, choices4[:len(cs_chapter.index)]))

choices5 = ['rgba(166,206,227, 0.8)', 'rgba(166,206,227, 0.8)', 'rgba(166,206,227, 0.8)', 'rgba(166,206,227, 0.8)'
           , 'rgba(166,206,227, 0.8)', 'rgba(166,206,227, 0.8)', 'rgba(166,206,227, 0.8)', 'rgba(166,206,227, 0.8)'
           , 'rgba(166,206,227, 0.8)', 'rgba(166,206,227, 0.8)', 'rgba(166,206,227, 0.8)', 'rgba(166,206,227, 0.8)'
           , 'rgba(166,206,227, 0.8)', 'rgba(166,206,227, 0.8)', 'rgba(166,206,227, 0.8)', 'rgba(166,206,227, 0.8)'
           , 'rgba(166,206,227, 0.8)', 'rgba(166,206,227, 0.8)', 'rgba(166,206,227, 0.8)']
# create a dictionary to map colors to chapters, for dataset of full videos lengths
colors_chapter5 = dict(zip(cs_chapter.index, choices5[:len(cs_chapter.index)]))

#### Video engagement

In [ ]:
# x-axis needs to start from 1, videos.length and videos.length are normalized so that maximum=100
# hoverinfo => question_name: actual value
if videos is not None:
    trace1 = go.Bar(x = videos.index+1, y = videos.length/60,
                    text=['{0}: {1}%'.format(name.encode('utf-8'), value) 
                          for name, value in zip(videos.name, videos.length/videos.mmm*100)], hoverinfo='text',
                    marker=dict(color='rgba(202,178,214, 0.8)'), name='video_length')
    '''trace2 = go.Bar(x = videos.index+1, y = -videos.nstudents/(videos.nstudents.max()/100), 
                   text=['{0}: {1}'.format(name.encode('utf-8'), value) 
                         for name, value in zip(videos.name, videos.nstudents)], hoverinfo='text',
                    marker=dict(color=videos.chapter.map(colors_chapter)), name='nstudents_watched')'''
    trace2 = go.Bar(x = videos.index+1, y = (videos.mmm-videos.length)/60,
                    text=['{0}: {1}%'.format(name.encode('utf-8'), value) 
                          for name, value in zip(videos.name, itertools.repeat('100'))], hoverinfo='text',
                    marker=dict(color='rgba(106,61,154, 0.8)'), name='video_length', orientation = 'v',)
    
    data = [trace1, trace2]
    layout = go.Layout(barmode='stack', xaxis=dict(showticklabels=False), title='',
                       yaxis=dict(showticklabels=True, showgrid=False, title='video length (minutes)'), 
                       height=400, width=850, margin=go.Margin(t=50, b=50), showlegend=False)
    fig = go.Figure(data=data, layout=layout)
    print('\n\n' + 'What percentage of each video did learners watch?' + '\n')
    print('This graph shows the mean (across all students) percentage of a given video watched, across all videos. For more information, please see below:')
    config = {"displayModeBar": True, "showLink": False, "displaylogo": False, "modeBarButtonsToRemove": ['sendDataToCloud', 'zoomIn2d', 'zoomOut2d', 'autoScale2d', 'resetScale2d', 'hoverClosestCartesian', 'hoverCompareCartesian']}
    plot_html, plotdivid, width, height = py.offline._plot_html(fig, config, True, 525, 525, False)
    display(HTML(plot_html))
    display(HTML('<a href="#Table-of-Contents" style="float: left;">Back to top</a><a href="#" style="float: right;">Read more...</a>'))
else:
    display(HTML('There are no videos in this course.'))

### Graded problem activity

#### Graded problem engagement

In [ ]:
# x-axis needs to start from 1, problems.nstudents is normalized so that maximum=100 (the same as pct_correct)
# hoverinfo: question_name: actual value
pct_correct = (problems.ncorrect/problems.nstudents*100).round(2)
trace1 = go.Bar(x=problems.index+1, y = pct_correct, 
                text=['{0}<br>{1}: {2}%'.format(section.encode('utf-8'), name.encode('utf-8'), value) 
                      for section, name, value in zip(problems.section_name, problems.name, pct_correct)],
                hoverinfo='text', marker=dict(color='rgba(166,206,227, 0.8)'), name='pct_correct')
trace2 = go.Bar(x=problems.index+1, y = 100-pct_correct, 
                text=['{0}<br>{1}: {2}%'.format(section.encode('utf-8'), name.encode('utf-8'), value) 
                      for section, name, value in zip(problems.section_name, problems.name, pct_correct)],
                hoverinfo='text', marker=dict(color='rgba(166,206,227, 0.8)'), name='pct_correct')

data = [trace1]
layout = go.Layout(barmode='stack', xaxis=dict(showticklabels=False), title='',
                   yaxis=dict(showticklabels=True, showgrid=False, title='success rate'), 
                   height=440, width=850, margin=go.Margin(t=25, b=25), showlegend=False)
fig = go.Figure(data=data, layout=layout)
print('\n\n' + 'What percentage of learners answered each problem correctly on the first attempt?' + '\n')
print('This graph shows the percentage of learners who answered a given problem correctly on their first attempt, across all problems. For more information, please see below:')
config = {"displayModeBar": True, "showLink": False, "displaylogo": False, "modeBarButtonsToRemove": ['sendDataToCloud', 'zoomIn2d', 'zoomOut2d', 'autoScale2d', 'resetScale2d', 'hoverClosestCartesian', 'hoverCompareCartesian']}
plot_html, plotdivid, width, height = py.offline._plot_html(fig, config, True, 525, 525, False)
display(HTML(plot_html))
display(HTML('<a href="#Table-of-Contents" style="float: left;">Back to top</a><a href="#" style="float: right;">Read more...</a>'))

#### Self-test problem engagement

In [ ]:
# x-axis needs to start from 1, problems.nstudents is normalized so that maximum=100 (the same as pct_correct)
# hoverinfo: question_name: actual value
pct_correct2 = (self_test.ncorrect/self_test.nstudents*100).round(2)
trace1 = go.Bar(x=self_test.index+1, y = pct_correct2, 
                text=['<br>{0}: {1}%'.format(name.encode('utf-8'), value) 
                      for name, value in zip(self_test.name, pct_correct2)],
                hoverinfo='text', marker=dict(color="rgb(178,223,138, 0.8)"), name='pct_correct')

data = [trace1]
layout = go.Layout(barmode='stack', xaxis=dict(showticklabels=False), title='',
                   yaxis=dict(showticklabels=True, showgrid=False, title='success rate'), 
                   height=440, width=850, margin=go.Margin(t=25, b=25), showlegend=False)
fig = go.Figure(data=data, layout=layout)
print('\n\n' + 'What percentage of learners answered each problem correctly on the first attempt?' + '\n')
print('This graph shows the percentage of learners who answered a given problem correctly on their first attempt, across all problems. For more information, please see below:')
config = {"displayModeBar": True, "showLink": False, "displaylogo": False, "modeBarButtonsToRemove": ['sendDataToCloud', 'zoomIn2d', 'zoomOut2d', 'autoScale2d', 'resetScale2d', 'hoverClosestCartesian', 'hoverCompareCartesian']}
plot_html, plotdivid, width, height = py.offline._plot_html(fig, config, True, 525, 525, False)
display(HTML(plot_html))
display(HTML('<a href="#Table-of-Contents" style="float: left;">Back to top</a><a href="#" style="float: right;">Read more...</a>'))

In [ ]:
def query_least(course_id=course_id):
    """
    Given the course_id, return the distribution of answers for the 10 least successful graded_problems
    """
    least = "', '".join(problems.loc[pct_correct.argsort()[:5], 'problem_id'].values)
    # query 10 least successful problems and corresponding responses by each student
    query = """
    Select item.answer_id As problem_id, user_id, item.response As response, item.correctness
    From [{0}.problem_analysis]
    Where item.answer_id in ('{1}')
    """.format(course_id, least)

    answers = pd.io.gbq.read_gbq(query, project_id='yourprojectname', verbose=False)
    answers = answers.merge(problems[['name', 'problem_id']])
    # the correct answers: most common responses
    correct =  answers[answers.item_correctness=='correct'].groupby('name').response.first().to_dict()
    # total # of responses
    count = answers.groupby('name').response.count()
    
    answers.response = answers.response.apply(lambda x: x.replace('[', '').replace(']', '').split(', '))
    rows = []
    # explode items in a list to multiple rows <= those with more than 1 answer
    _ = answers.apply(lambda row: [rows.append([row['name'], row['user_id'], choice]) 
                             for choice in row.response], axis=1)
    answers_new = pd.DataFrame(rows, columns=['name', 'user_id', 'response'])#.set_index(['name', 'opponent'])
    # aggregate and calculate % distribution of answers for each problem
    if not count.empty:
        answers_pct = answers_new.groupby('name').response.value_counts().unstack('name').divide(count, axis=1)
    else:
        answer_pct = None
    return correct, answers_pct#, answers

#### Ten graded problems with lowest success rate

In [ ]:
if course_id in not_mc_list:
    pass
else:
    correct, answers_pct = query_least()
    if answers_pct is not None:    
        def correct_color(col):
            # green for correct answers, blue otherwise
            return ['rgb(31,120,180)' if x in correct[answers_pct.columns[col]] else 'rgb(166,206,227)' for x in answers_pct.index]

        fig = tls.make_subplots(rows=1, cols=5, print_grid=False, vertical_spacing=0.25)
        # the first five problems
        for i in range(5):
            quesion = answers_pct.iloc[:, i].dropna()*100
            fig.append_trace(go.Bar(x=quesion.index, y=quesion, name=answers_pct.columns[i], 
                                    marker=dict(color=correct_color(i)), showlegend=False), 1, i+1)    

        for i in range(1, answers_pct.shape[1]+1):
            fig['layout']['xaxis%s' % i].update(tickangle=45, tickfont=dict(size=8),
                                                title=answers_pct.columns[i-1], titlefont=dict(size=10))
            fig['layout']['yaxis%s' % i].update(showgrid=False, tickfont=dict(size=8), title='% of answers', range=[0,100])

            fig['layout'].update(height=500, width=850, 
                                 title = '')  
        print('\n\n' + 'What problems did learners struggle with most?' + '\n')
        print('Each graph represents a single problem, and shows the percentage of initial responses for each possible answer.' + '\n' + 'The five problems' +
          ' with the lowest success rate are shown. For more information, please see below:')    
        config = {"displayModeBar": True, "showLink": False, "displaylogo": False, "modeBarButtonsToRemove": ['sendDataToCloud', 'zoomIn2d', 'zoomOut2d', 'autoScale2d', 'resetScale2d', 'hoverClosestCartesian', 'hoverCompareCartesian']}
        plot_html, plotdivid, width, height = py.offline._plot_html(fig, config, True, 525, 525, False)
        display(HTML(plot_html))
        display(HTML('<a href="#Table-of-Contents" style="float: left;">Back to top</a><a href="#" style="float: right;">Read more...</a>'))

### Overall engagement:
** Learner type **
- Registered: learners registered in the course
- Sampled: learners who accessed the course at least once
- Involved: learners with sum_dt > 15 min
- Passed: learners whose grade is at least 50%
- Verified: those purchased the verified certificate

** sum_dt **: Total elapsed time spent by learner on this course, based on time difference between consecutive events, with a 5 min max cutoff, based on event data

In [ ]:
def query_pc(course_id = course_id):     
    """
    Given course_id(e.g. 'UBCx__Marketing1x__3T2015'), query and calculate ndays_act, sum_dt, nforum_posts, 
    nvideos_watched, nproblems_attempted, pct_video_watched, pct_problem_attempted for involved students,
    also return total_videos and total_problems.
    """
    query = """
    Select pc.user_id As user_id, pc.course_id As course_id, pc.mode As mode, pc.grade As grade, 
    pc.ndays_act As ndays_act, pc.sum_dt As sum_dt, pc.nforum_posts As nforum_posts,
    v.videos_watched As nvideos_watched, p.problems_attempted As nproblems_attempted
    From [{0}.person_course] pc
    Left Join
    (SELECT username, exact_count_distinct(video_id) As videos_watched 
    FROM [{0}.video_stats_day]
    Group By username) v
    on pc.username = v.username
    Left Join 
    (Select user_id, exact_count_distinct(item.answer_id) As problems_attempted
    From [{0}.problem_analysis]
    Group By user_id) p
    On pc.user_id = p.user_id
    Where pc.sum_dt > 0""".format(course_id)
    total_videos = 0
    try:
        df = pd.io.gbq.read_gbq(query, project_id='yourprojectname', verbose=False).fillna(0)
        total_videos = min(df.nvideos_watched.max(), cs_chapter.video.sum())
        df['pct_video_watched'] = df.nvideos_watched / total_videos        
    except:      
        vidya_query = """
        Select pc.user_id As user_id, pc.course_id As course_id, pc.mode As mode, pc.grade As grade, 
        pc.ndays_act As ndays_act, pc.sum_dt As sum_dt, pc.nforum_posts As nforum_posts,
        p.problems_attempted As nproblems_attempted
        From [{0}.person_course] pc
        Left Join 
        (Select user_id, exact_count_distinct(item.answer_id) As problems_attempted
        From [{0}.problem_analysis]
        Group By user_id) p
        On pc.user_id = p.user_id
        Where pc.sum_dt > 0""".format(course_id)
        df = pd.io.gbq.read_gbq(vidya_query, project_id='yourprojectname', verbose=False).fillna(0)
        total_videos = 0
        df['pct_video_watched'] = 0
    # course_axis includes items not accessible to the students, 
    # => total_videos/total_problems are maximum number of videos/problems students accessed 
    # if smaller than the number from course_axis then use the latter one

    
    total_problems = min(df.nproblems_attempted.max(), 
                         cs_chapter.graded_problem.sum() + cs_chapter.self_test.sum() if 'self_test' in cs_chapter.columns and 'graded_problem' in cs_chapter.columns
                         else cs_chapter.self_test.sum() if 'self_test' in cs_chapter.columns
                         else cs_chapter.graded_problem.sum())
    df['pct_problem_attempted'] = df.nproblems_attempted / total_problems
    
    
    return  total_videos, total_problems, df

In [ ]:
def compute_srp(pc):
    # pc for the sampled
    pc_sampled = pc.copy()
    # pc for the involved
    pc_learned = pc[pc.sum_dt>900].copy()
    # pc for the passed
    pc_passed = pc[pc.grade>=0.5].copy()
    # added category column
    pc_sampled['category'] = 'Sampled'
    pc_learned['category'] = 'Involved'
    pc_passed['category'] = 'Passed'
    srp = pd.concat([pc_sampled, pc_learned, pc_passed])
    # aggregate nforum_posts => # students posted, others by median, by learner type
    try:
        srp_agg = srp.groupby('category').agg({'nvideos_watched': np.median, 
                                 'nproblems_attempted': np.median, 'ndays_act': np.median, 
                                 'sum_dt': np.median, 'nforum_posts': lambda x: (x > 0).sum()})
    except:
        srp_agg = srp.groupby('category').agg({ 
                                 'nproblems_attempted': np.median, 'ndays_act': np.median, 
                                 'sum_dt': np.median, 'nforum_posts': lambda x: (x > 0).sum()})        
    srp_agg = srp_agg.reindex(index = ['Sampled', 'Involved', 'Passed'])
    
    return srp_agg

In [ ]:
total_videos, total_problems, pc = query_pc()
# convert to %
pc[['pct_video_watched', 'pct_problem_attempted']] = \
pc[['pct_video_watched', 'pct_problem_attempted']].applymap(lambda x: "{0:.2f}".format(x * 100))
srp_agg = compute_srp(pc)

In [ ]:
def plot_pls(df, course_id=course_id, title=None):
    """
    Plot students' activity: median sum_dt, median ndays_act, # students posted, 
    nproblems_attempted, nvideos_watched, grouped by passed vs. involved vs. sampled;
    """
    if course_id in pe_list:
        query = \
        """SELECT Count(*) As Registered, 
        Sum(Case When sum_dt > 0 Then 1 Else 0 End) As Sampled,
        Sum(Case When sum_dt > 900 Then 1 Else 0 End) As Involved, 
        Sum(Case When grade >= 0.5 Then 1 Else 0 End) As Passed
        FROM [%s.person_course]""" % course_id
    else:
        query = \
        """SELECT Count(*) As Registered, 
        Sum(Case When sum_dt > 0 Then 1 Else 0 End) As Sampled,
        Sum(Case When sum_dt > 900 Then 1 Else 0 End) As Involved, 
        Sum(Case When grade >= 0.5 Then 1 Else 0 End) As Passed, 
        Sum(Case When mode='verified' Then 1 Else 0 End) As Verified
        FROM [%s.person_course]""" % course_id
    stats =  pd.io.gbq.read_gbq(query, project_id='yourprojectname', verbose=False)
    # print stats

    trace1 = go.Bar(x=stats.values[0], y=stats.columns, orientation='h', showlegend=False, name='# of conversion')
    if 'nvideos_watched' in df:
        trace2 = go.Bar(x=df.index, y=df.nvideos_watched, showlegend=True, name='nvideos watched')
    trace3 = go.Bar(x=df.index, y=df.nproblems_attempted, showlegend=True, name='nproblems attempted')
    trace4 = go.Bar(x=df.index, y=df.sum_dt/3600, showlegend=False, name='median sum_dt (H)')
    trace5 = go.Bar(x=df.index, y=df.ndays_act, showlegend=False, name='median days active')
    trace6 = go.Bar(x=df.index, y=df.nforum_posts, showlegend=False, name='# students posted')

    fig = tls.make_subplots(rows=1, cols=5, shared_xaxes=True, print_grid=False)
    fig.append_trace(trace1, 1, 1)
    if 'nvideos_watched' in df:
        fig.append_trace(trace2, 1, 5)
    fig.append_trace(trace3, 1, 5)
    fig.append_trace(trace4, 1, 2)
    fig.append_trace(trace5, 1, 3)
    fig.append_trace(trace6, 1, 4)


    fig['layout'].update(barmode='stack', height=300, width=900, margin=go.Margin(t=40), title=title)
    fig['layout']['legend'].update(font=dict(size=10))
    fig['layout']['xaxis1'].update(title='# students', showgrid=False,
                                  titlefont=dict(size=12), tickfont=dict(size=10))
    fig['layout']['xaxis2'].update(title='median sum_dt (H)', showgrid=False,
                                  titlefont=dict(size=12), tickfont=dict(size=10))
    fig['layout']['xaxis3'].update(title='median days active', showgrid=False,
                                  titlefont=dict(size=12), tickfont=dict(size=10))
    fig['layout']['xaxis4'].update(title='# students posted', showgrid=False,
                                  titlefont=dict(size=12), tickfont=dict(size=10))
    fig['layout']['xaxis5'].update(title='median events', showgrid=False, 
                                   titlefont=dict(size=12), tickfont=dict(size=10))
    fig['layout']['yaxis1'].update(autorange='reversed', showgrid=False, tickfont=dict(size=10))
    fig['layout']['yaxis2'].update(showgrid=False, tickfont=dict(size=10))
    fig['layout']['yaxis3'].update(showgrid=False, tickfont=dict(size=10))
    fig['layout']['yaxis4'].update(showgrid=False, tickfont=dict(size=10))
    fig['layout']['yaxis5'].update(showgrid=False, tickfont=dict(size=10))

    config = {"displayModeBar": True, "showLink": False, "displaylogo": False, "modeBarButtonsToRemove": ['sendDataToCloud', 'zoomIn2d', 'zoomOut2d', 'autoScale2d', 'resetScale2d', 'hoverClosestCartesian', 'hoverCompareCartesian']}
    plot_html, plotdivid, width, height = py.offline._plot_html(fig, config, True, 525, 525, False)
    display(HTML(plot_html))

In [ ]:
plot_pls(srp_agg, title="Students' engagement: Sampled vs. Involved vs. Passed")

In [ ]:
# pc for the involved
pc_activity = pc[pc.sum_dt>900].copy()
# density map
trace2 = go.Histogram2d(x=pc_activity.pct_video_watched, y=pc_activity.pct_problem_attempted,
            histnorm='probability',
            autobinx=False,
            xbins=dict(start=0, end=100, size=10),
            autobiny=False,
            ybins=dict(start=0, end=100, size=10),
            colorscale=[[0, 'rgb(255,255,255)'], [1/1000, 'rgb(255,255,255)'], [1/100, 'rgb(242,211,56)'], 
                            [1/10, 'rgb(242,143,56)'], [1, 'rgb(217,30,30)']],
            zsmooth='false')
# illustration on how to read the density map
z = [[1, 0.5, 0.5, 0.5], [0.5, 0.5, 0, 0], [0.5, 0, 0.5, 0], [0.5, 0, 0, 1]]
z_text = [['Early dropout', '', 'Videos only', ''], ['', 'Progress', '', ''],
         ['Problems only', '', 'Progress', ''], ['', '', '', 'Completed']]
annotations = []
for n, row in enumerate(z):
    for m, val in enumerate(row):
        text = z_text[n][m]
        annotations.append(
            dict(
                text=str(text),
                x=m, y=n,
                font=dict(color='black'),
                showarrow=False)
            )
colorscale=[[0, 'rgb(82,82,82)'], [0.5, 'rgb(150,150,150)'], [1, 'rgb(204,204,204)']]
trace1 = go.Heatmap(z=z, colorscale=colorscale, showscale=False, hoverinfo='none')

fig = tls.make_subplots(rows=1, cols=2, print_grid=False)
fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)

fig['layout']['xaxis1'].update(ticks='', showticklabels=False, showgrid=False, title='% videos')
fig['layout']['xaxis2'].update(title='% videos (total:{0})'.format(int(total_videos)))
fig['layout']['yaxis1'].update(ticks='', showticklabels=False, showgrid=False, zeroline=False, title='% problems')
fig['layout']['yaxis2'].update(title='% problems (total:{0})'.format(int(total_problems)))
fig['layout'].update(
    width=850, height=400, annotations=annotations,
    title='Density map illustrating pattern of video and problem activity for the involved',
    margin=go.Margin(l=40, t=40)
)

config = {"displayModeBar": True, "showLink": False, "displaylogo": False, "modeBarButtonsToRemove": ['sendDataToCloud', 'zoomIn2d', 'zoomOut2d', 'autoScale2d', 'resetScale2d', 'hoverClosestCartesian', 'hoverCompareCartesian']}
plot_html, plotdivid, width, height = py.offline._plot_html(fig, config, True, 525, 525, False)
display(HTML(plot_html))